# G1 scraping
This notebook aims to build a scaper of the news in https://g1.globo.com/

## Setup

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.common.by import By

# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

## Start

In [2]:
driver.get('https://g1.globo.com/')

In [3]:
# scrolling the page

scrolls = 10

current_height = driver.execute_script("return document.body.scrollHeight")

for i in range(scrolls):
    # scroll to the end of the page
    driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")

    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # if not autoscoll
    if new_height == current_height:
        # click the 'Veja mais' button
        driver.find_element(By.CSS_SELECTOR, value='.load-more > a:nth-child(1)').click()
        
        # repeat the scroll
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
        current_height = driver.execute_script("return document.body.scrollHeight")
    else:
        current_height = new_height
        
    print(f'Scroll {i + 1}, height={current_height}')
    
    # time for the page to load
    sleep(4)

Scroll 1, height=7941
Scroll 2, height=14557
Scroll 3, height=14294
Scroll 4, height=14101
Scroll 5, height=17401
Scroll 6, height=17015
Scroll 7, height=20202
Scroll 8, height=19816
Scroll 9, height=22902
Scroll 10, height=22516
